# GP-VAE Training on COIL-100 (Standard Task) - FullRank Kernel

This notebook trains **GP-VAE** on COIL-100 dataset using the **FullRank kernel** with **early stopping**.

## Kernel: FullRank
- **Nonparametric kernel** with full covariance matrix
- K = LL^T where L is learned directly
- **O(Q²) parameters** where Q = number of views
- **Best for**: Maximum flexibility, capturing any covariance structure

## Dataset Info:
- **COIL-100**: 100 objects × 18 views (every 20°: 0°, 20°, ..., 340°)
- **Image size**: 128×128×3 RGB
- **Task**: Standard split (random train/val/test)

## Prerequisites:
- ✅ Trained VAE weights from `train_vae_colab_standard.ipynb`
- ✅ COIL-100 data file: `data/coil100/coil100_task1_standard.h5`

## 1. Check GPU Availability

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not detected! Go to Runtime → Change runtime type → GPU")

: 

## 2. Auto-Detect Project Path

In [ ]:
import os
import sys

current_dir = os.getcwd()
print(f"📍 Current directory: {current_dir}")

# Task configuration
DATA_TASK = "task1_standard"
KERNEL_TYPE = "full_rank"

if current_dir == '/content':
    print("\n🔄 Mounting Google Drive...")
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_path = '/content/drive/MyDrive/gppvae'
        if os.path.exists(drive_path):
            PROJECT_PATH = drive_path
            print(f"✅ Found project in Google Drive: {PROJECT_PATH}")
        else:
            print(f"⚠️ Project not found at: {drive_path}")
            PROJECT_PATH = '/content'
    except Exception as e:
        print(f"Could not mount Drive: {e}")
        PROJECT_PATH = '/content'
else:
    if 'notebooks' in current_dir:
        PROJECT_PATH = os.path.dirname(current_dir)
    else:
        PROJECT_PATH = current_dir
    print(f"💻 Using project path: {PROJECT_PATH}")

# Verify structure
print(f"\n📁 Contents of {PROJECT_PATH}:")
if os.path.exists(PROJECT_PATH):
    for item in sorted(os.listdir(PROJECT_PATH))[:10]:
        item_path = os.path.join(PROJECT_PATH, item)
        marker = "📂" if os.path.isdir(item_path) else "📄"
        print(f"   {marker} {item}")

# Check required files
print(f"\n🔍 Checking required files:")
data_path = os.path.join(PROJECT_PATH, f'data/coil100/coil100_{DATA_TASK}.h5')
required = {
    'GPPVAE code': os.path.exists(os.path.join(PROJECT_PATH, 'GPPVAE')),
    'COIL-100 data': os.path.exists(data_path),
}

# Check for VAE runs
vae_base_dir = os.path.join(PROJECT_PATH, f'out/vae_colab_{DATA_TASK}_standard')
vae_run_found = False
if os.path.exists(vae_base_dir):
    runs = [d for d in os.listdir(vae_base_dir) if os.path.isdir(os.path.join(vae_base_dir, d))]
    for run in runs:
        weights_dir = os.path.join(vae_base_dir, run, 'weights')
        if os.path.exists(weights_dir) and any(f.endswith('.pt') for f in os.listdir(weights_dir)):
            vae_run_found = True
            break
required['VAE weights'] = vae_run_found

for name, exists in required.items():
    status = "✅" if exists else "❌"
    print(f"   {status} {name}")

if not required['VAE weights']:
    print(f"\n⚠️ No trained VAE found! Run train_vae_colab_standard.ipynb first.")

## 3. Install Dependencies

In [ ]:
!pip install -q wandb==0.12.21 imageio==2.15.0 pyyaml

import wandb
import imageio
import yaml
import numpy as np
print("✅ All dependencies installed successfully!")

## 4. Login to W&B (Optional)

In [ ]:
import wandb
wandb.login()

# Or run offline:
# import os
# os.environ['WANDB_MODE'] = 'offline'

## 5. Navigate to Project Directory

In [ ]:
import os
import sys

os.chdir(PROJECT_PATH)
print(f"Current directory: {os.getcwd()}")

# Add to Python path
sys.path.insert(0, os.path.join(PROJECT_PATH, 'GPPVAE/pysrc/coil100'))

print("\nProject structure:")
!ls -la

## 6. Find VAE Weights

In [ ]:
import os
import pickle

vae_base_dir = f'./out/vae_colab_{DATA_TASK}_standard'
vae_runs = []

if os.path.exists(vae_base_dir):
    for run_dir in sorted(os.listdir(vae_base_dir), reverse=True):
        run_path = os.path.join(vae_base_dir, run_dir)
        cfg_path = os.path.join(run_path, 'vae.cfg.p')
        weights_dir = os.path.join(run_path, 'weights')
        
        if os.path.exists(cfg_path) and os.path.exists(weights_dir):
            weight_files = sorted([f for f in os.listdir(weights_dir) if f.endswith('.pt')])
            if weight_files:
                vae_runs.append({
                    'run_dir': run_dir,
                    'cfg_path': cfg_path,
                    'weights_dir': weights_dir,
                    'weight_files': weight_files
                })

if vae_runs:
    print(f"✅ Found {len(vae_runs)} VAE training run(s):\n")
    for i, run in enumerate(vae_runs[:3], 1):
        print(f"Run {i}: {run['run_dir']}")
        vae_cfg = pickle.load(open(run['cfg_path'], 'rb'))
        print(f"   Config: zdim={vae_cfg.get('zdim', 'N/A')}, nf={vae_cfg.get('nf', 'N/A')}")
        print(f"   Checkpoints: {len(run['weight_files'])} files")
        print(f"   Latest: {run['weight_files'][-1]}")
    
    # Recommend latest
    latest = vae_runs[0]
    print(f"\n💡 Recommendation: Use {latest['run_dir']}")
    print(f"   VAE_CFG = '{latest['cfg_path']}'")
    print(f"   VAE_WEIGHTS = '{os.path.join(latest['weights_dir'], latest['weight_files'][-1])}'")
else:
    print("❌ No VAE runs found! Train VAE first.")

## 7. Configure Training

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# ============================================================================
# UPDATE THESE PATHS based on Cell 6 output!
# ============================================================================
VAE_CFG = './out/vae_colab_task1_standard_standard/YYYYMMDD_HHMMSS/vae.cfg.p'  # UPDATE THIS
VAE_WEIGHTS = './out/vae_colab_task1_standard_standard/YYYYMMDD_HHMMSS/weights/weights.00499.pt'  # UPDATE THIS

CONFIG = {
    # Data
    'data': f'./data/coil100/coil100_{DATA_TASK}.h5',
    'outdir': f'./out/gppvae_coil100_{KERNEL_TYPE}_{DATA_TASK}/{timestamp}',
    
    # VAE (pre-trained)
    'vae_cfg': VAE_CFG,
    'vae_weights': VAE_WEIGHTS,
    
    # Training (increased for early stopping)
    'epochs': 1200,
    'batch_size': 64,
    'vae_lr': 0.001,
    'gp_lr': 0.001,
    'xdim': 64,  # Object embedding dimension
    
    # Kernel - FullRank
    'view_kernel': KERNEL_TYPE,
    'kernel_kwargs': {},  # FullRank has no hyperparameters
    
    # Logging
    'epoch_cb': 100,
    'use_wandb': True,
    'wandb_project': 'gppvae-coil100',
    'wandb_run_name': f'gppvae_{KERNEL_TYPE}_{DATA_TASK}_{timestamp}',
    'seed': 0,
}

print("GP-VAE Training Configuration:")
print("=" * 60)
for key, value in CONFIG.items():
    print(f"  {key:20s}: {value}")
print("=" * 60)

if not os.path.exists(CONFIG['vae_weights']):
    print(f"\n⚠️ Update VAE_CFG and VAE_WEIGHTS paths based on Cell 6 output!")

## 8. Import Modules

In [ ]:
# IMPORTANT: Add coil100 to path FIRST before importing anything
# This ensures coil100's data_parser is used, not faceplace's
import sys
import os

coil100_path = os.path.join(PROJECT_PATH, 'GPPVAE/pysrc/coil100')
if coil100_path not in sys.path:
    sys.path.insert(0, coil100_path)

os.chdir(coil100_path)
print(f"Working directory: {os.getcwd()}")

import matplotlib
matplotlib.use('Agg')

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from vae import FaceVAE
from vmod import Vmodel
from gp import GP
import numpy as np
import logging
import pylab as pl
from utils import smartSum, smartAppendDict, smartAppend, export_scripts
from callbacks import callback_gppvae
import pickle
import time
import wandb

# COIL-100 data parser (explicitly import from coil100, not faceplace)
from data_parser import COIL100Dataset, get_n_views, get_num_objects

# Verify we're using the right data_parser
import data_parser
print(f"✅ data_parser loaded from: {data_parser.__file__}")
print("✅ All modules imported successfully!")

## 9. Setup Training Environment

In [ ]:
os.chdir(PROJECT_PATH)

# Create output directories
outdir = CONFIG['outdir']
wdir = os.path.join(outdir, "weights")
fdir = os.path.join(outdir, "plots")
os.makedirs(wdir, exist_ok=True)
os.makedirs(fdir, exist_ok=True)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Logging
log_format = "%(asctime)s %(message)s"
logging.basicConfig(level=logging.INFO, format=log_format, datefmt="%m/%d %I:%M:%S %p")
fh = logging.FileHandler(os.path.join(outdir, "log.txt"))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

# Copy scripts
export_scripts(os.path.join(outdir, "scripts"))

print(f"✅ Output directory: {outdir}")

## 10. Initialize Models and Data

In [ ]:
torch.manual_seed(CONFIG['seed'])

# Initialize W&B
if CONFIG['use_wandb']:
    wandb.init(
        project=CONFIG['wandb_project'],
        name=CONFIG['wandb_run_name'],
        config=CONFIG
    )

# Load VAE config
vae_cfg = pickle.load(open(CONFIG['vae_cfg'], "rb"))
print(f"VAE config: {vae_cfg}")

# Load pre-trained VAE
print("\nLoading pre-trained VAE...")
vae = FaceVAE(**vae_cfg).to(device)
vae_state = torch.load(CONFIG['vae_weights'], map_location=device)
vae.load_state_dict(vae_state)
print(f"✅ VAE loaded from {CONFIG['vae_weights']}")

# Load COIL-100 data
print("\nLoading COIL-100 data...")
train_data = COIL100Dataset(CONFIG['data'], split='train', use_angle_encoding=False)
val_data = COIL100Dataset(CONFIG['data'], split='val', use_angle_encoding=False)

train_queue = DataLoader(train_data, batch_size=CONFIG['batch_size'], shuffle=True)
val_queue = DataLoader(val_data, batch_size=CONFIG['batch_size'], shuffle=False)

# Get P and Q
# IMPORTANT: Use get_num_objects to get correct P (includes all objects from all splits)
# This ensures embedding indices don't go out of bounds
P = get_num_objects(CONFIG['data'])  # 100 for COIL-100
Q = get_n_views()  # 18
print(f"\nP (objects): {P}")
print(f"Q (views): {Q}")

# Create object and view tensors (Did and Rid are 1D tensors)
Dt = Variable(train_data.Did.long(), requires_grad=False).to(device)
Dv = Variable(val_data.Did.long(), requires_grad=False).to(device)
Wt = Variable(train_data.Rid.long(), requires_grad=False).to(device)
Wv = Variable(val_data.Rid.long(), requires_grad=False).to(device)

print(f"\nTrain views: {torch.unique(Wt).cpu().numpy()}")
print(f"Val views: {torch.unique(Wv).cpu().numpy()}")

# Initialize Vmodel with FullRank kernel
print(f"\n🔬 Initializing Vmodel with '{KERNEL_TYPE}' kernel...")
vm = Vmodel(
    P=P, Q=Q, p=CONFIG['xdim'],
    view_kernel=CONFIG['view_kernel'],
    **CONFIG['kernel_kwargs']
).to(device)

gp = GP(n_rand_effs=1).to(device)

# Combine GP parameters
gp_params = nn.ParameterList()
gp_params.extend(vm.parameters())
gp_params.extend(gp.parameters())

print(f"\n✅ Models initialized:")
print(f"   VAE parameters: {sum(p.numel() for p in vae.parameters()):,}")
print(f"   Vmodel parameters: {sum(p.numel() for p in vm.parameters()):,}")
print(f"   GP parameters: {sum(p.numel() for p in gp.parameters()):,}")

# Optimizers
vae_optimizer = optim.Adam(vae.parameters(), lr=CONFIG['vae_lr'])
gp_optimizer = optim.Adam(gp_params, lr=CONFIG['gp_lr'])

## 11. Define Training Functions

In [ ]:
def encode_Y(vae, train_queue):
    """Encode all training images to get latent codes"""
    vae.eval()
    with torch.no_grad():
        n = train_queue.dataset.Y.shape[0]
        Zm = Variable(torch.zeros(n, vae_cfg["zdim"]), requires_grad=False).to(device)
        Zs = Variable(torch.zeros(n, vae_cfg["zdim"]), requires_grad=False).to(device)
        
        for batch_i, data in enumerate(train_queue):
            y = data[0].to(device)
            idxs = data[-1].to(device)
            zm, zs = vae.encode(y)
            Zm[idxs], Zs[idxs] = zm.detach(), zs.detach()
    return Zm, Zs


def eval_step(vae, gp, vm, val_queue, Zm, Vt, Vv, Wv):
    """Evaluation step"""
    rv = {}
    with torch.no_grad():
        _X = vm.x().data.cpu().numpy()
        _W = vm.v().data.cpu().numpy()
        covs = {"XX": np.dot(_X, _X.T), "WW": np.dot(_W, _W.T)}
        rv["vars"] = gp.get_vs().data.cpu().numpy()
        
        # Out-of-sample prediction
        vs = gp.get_vs()
        U, UBi, _ = gp.U_UBi_Shb([Vt], vs)
        Kiz = gp.solve(Zm, U, UBi, vs)
        Zo = vs[0] * Vv.mm(Vt.transpose(0, 1).mm(Kiz))
        
        mse_out = Variable(torch.zeros(Vv.shape[0], 1), requires_grad=False).to(device)
        mse_val = Variable(torch.zeros(Vv.shape[0], 1), requires_grad=False).to(device)
        
        all_Yv, all_Yr, all_Yo = [], [], []
        
        for batch_i, data in enumerate(val_queue):
            idxs = data[-1].to(device)
            Yv = data[0].to(device)
            Zv = vae.encode(Yv)[0].detach()
            Yr = vae.decode(Zv)
            Yo = vae.decode(Zo[idxs])
            mse_out[idxs] = ((Yv - Yo) ** 2).view(Yv.shape[0], -1).mean(1)[:, None].detach()
            mse_val[idxs] = ((Yv - Yr) ** 2).view(Yv.shape[0], -1).mean(1)[:, None].detach()
            
            all_Yv.append(Yv.data.cpu().numpy().transpose(0, 2, 3, 1))
            all_Yr.append(Yr.data.cpu().numpy().transpose(0, 2, 3, 1))
            all_Yo.append(Yo.data.cpu().numpy().transpose(0, 2, 3, 1))
        
        all_Yv = np.concatenate(all_Yv, axis=0)
        all_Yr = np.concatenate(all_Yr, axis=0)
        all_Yo = np.concatenate(all_Yo, axis=0)
        
        n_total = all_Yv.shape[0]
        sample_stride = max(1, n_total // 24)
        sample_indices = np.arange(0, n_total, sample_stride)[:24]
        
        imgs = {
            "Yv": all_Yv[sample_indices],
            "Yr": all_Yr[sample_indices],
            "Yo": all_Yo[sample_indices]
        }
        
        rv["mse_out"] = float(mse_out.data.mean().cpu())
        rv["mse_val"] = float(mse_val.data.mean().cpu())
    
    return rv, imgs, covs


def backprop_and_update(vae, gp, vm, train_queue, Dt, Wt, Eps, Zb, Vbs, vbs, vae_optimizer, gp_optimizer):
    """Joint optimization step"""
    rv = {}
    vae_optimizer.zero_grad()
    gp_optimizer.zero_grad()
    vae.train()
    gp.train()
    vm.train()
    
    for batch_i, data in enumerate(train_queue):
        y = data[0].to(device)
        eps = Eps[data[-1]]
        _d = Dt[data[-1]]
        _w = Wt[data[-1]]
        _Zb = Zb[data[-1]]
        _Vbs = [Vbs[0][data[-1]]]
        
        zm, zs = vae.encode(y)
        z = zm + zs * eps
        yr = vae.decode(z)
        recon_term, mse = vae.nll(y, yr)
        
        _Vs = [vm(_d, _w)]
        gp_nll_fo = gp.taylor_expansion(z, _Vs, _Zb, _Vbs, vbs) / vae.K
        pen_term = -0.5 * zs.sum(1)[:, None] / vae.K
        
        loss = (recon_term + gp_nll_fo + pen_term).sum()
        loss.backward()
        
        _n = train_queue.dataset.Y.shape[0]
        smartSum(rv, "mse", float(mse.data.sum().cpu()) / _n)
        smartSum(rv, "recon_term", float(recon_term.data.sum().cpu()) / _n)
        smartSum(rv, "pen_term", float(pen_term.data.sum().cpu()) / _n)
    
    vae_optimizer.step()
    gp_optimizer.step()
    
    return rv

print("✅ Training functions defined")

## 12. Train GP-VAE 🚀 (with Early Stopping)

In [ ]:
import time

history = {}
start_time = time.time()

# -----------------------------
# Early stopping configuration
# -----------------------------
early_stop_patience = 150        # epochs without improvement
early_stop_min_delta = 1e-4      # minimum improvement threshold

best_mse_out = float("inf")
best_epoch = -1
no_improve_epochs = 0

print(f"🚀 Training GP-VAE with {KERNEL_TYPE} kernel for up to {CONFIG['epochs']} epochs...")
print(f"🛑 Early stopping patience = {early_stop_patience}")
print("=" * 70)

for epoch in range(CONFIG['epochs']):
    epoch_start = time.time()

    # -------- Encode training data --------
    Zm, Zs = encode_Y(vae, train_queue)
    Eps = Variable(torch.randn(*Zs.shape), requires_grad=False).to(device)
    Z = Zm + Eps * Zs

    # -------- Precompute V --------
    Vt = vm(Dt, Wt).detach()
    Vv = vm(Dv, Wv).detach()

    # -------- Validation step --------
    rv_eval, imgs, covs = eval_step(
        vae, gp, vm, val_queue, Zm, Vt, Vv, Wv
    )

    # -------- GP Taylor expansion --------
    Zb, Vbs, vbs, gp_nll = gp.taylor_coeff(Z, [Vt])
    rv_eval["gp_nll"] = float(gp_nll.data.mean().cpu()) / vae.K

    # -------- Backprop --------
    rv_back = backprop_and_update(
        vae, gp, vm, train_queue, Dt, Wt,
        Eps, Zb, Vbs, vbs,
        vae_optimizer, gp_optimizer
    )

    rv_back["loss"] = (
        rv_back["recon_term"] +
        rv_eval["gp_nll"] +
        rv_back["pen_term"]
    )

    # -------- Logging --------
    smartAppendDict(history, rv_eval)
    smartAppendDict(history, rv_back)
    smartAppend(history, "vs", gp.get_vs().data.cpu().numpy())

    vs = gp.get_vs().data.cpu().numpy()
    variance_ratio = vs[0] / (vs[0] + vs[1])

    current_mse_out = rv_eval["mse_out"]

    # -------- Early stopping check --------
    if current_mse_out < best_mse_out - early_stop_min_delta:
        best_mse_out = current_mse_out
        best_epoch = epoch
        no_improve_epochs = 0

        # Save BEST checkpoint
        torch.save(
            vae.state_dict(),
            os.path.join(wdir, "vae_weights.best.pt")
        )
        torch.save(
            {'gp_state': gp.state_dict(), 'vm_state': vm.state_dict()},
            os.path.join(wdir, "gp_weights.best.pt")
        )

    else:
        no_improve_epochs += 1

    # -------- Console output --------
    if epoch % 5 == 0 or epoch == CONFIG['epochs'] - 1:
        print(
            f"Epoch {epoch:4d} | "
            f"MSE train: {rv_back['mse']:.6f} | "
            f"MSE out: {current_mse_out:.6f} | "
            f"GP NLL: {rv_eval['gp_nll']:.4f} | "
            f"v₀/(v₀+v₁): {variance_ratio:.3f}"
        )

    # -------- wandb --------
    if CONFIG['use_wandb']:
        log_dict = {
            "epoch": epoch,
            "mse_train": rv_back["mse"],
            "mse_out": current_mse_out,
            "gp_nll": rv_eval["gp_nll"],
            "variance_ratio": variance_ratio,
            "best_mse_out": best_mse_out,
            "no_improve_epochs": no_improve_epochs,
        }
        wandb.log(log_dict)

    # -------- Periodic checkpoint + plots --------
    if epoch % CONFIG['epoch_cb'] == 0 or epoch == CONFIG['epochs'] - 1:
        torch.save(
            vae.state_dict(),
            os.path.join(wdir, f"vae_weights.{epoch:05d}.pt")
        )
        torch.save(
            {'gp_state': gp.state_dict(), 'vm_state': vm.state_dict()},
            os.path.join(wdir, f"gp_weights.{epoch:05d}.pt")
        )
        ffile = os.path.join(fdir, f"plot.{epoch:05d}.png")
        callback_gppvae(epoch, history, covs, imgs, ffile)
        if CONFIG['use_wandb']:
            wandb.log({"reconstructions": wandb.Image(ffile)})
        print("  ✓ Checkpoint saved")

    # -------- Stop condition --------
    if no_improve_epochs >= early_stop_patience:
        print(
            f"\n⏹ Early stopping triggered at epoch {epoch}\n"
            f"   Best epoch: {best_epoch}\n"
            f"   Best mse_out: {best_mse_out:.6f}"
        )
        break

print(
    f"\n✅ Training complete in {(time.time()-start_time)/60:.1f} min\n"
    f"   Best epoch: {best_epoch}\n"
    f"   Best mse_out: {best_mse_out:.6f}"
)

if CONFIG['use_wandb']:
    wandb.finish()

## 13. View Results

In [ ]:
from IPython.display import Image, display
import glob

plot_files = sorted(glob.glob(os.path.join(fdir, "*.png")))
if plot_files:
    print(f"Latest plot: {plot_files[-1]}")
    display(Image(filename=plot_files[-1]))
else:
    print("No plots generated yet")

## 14. Download Results

In [ ]:
!zip -r /content/gppvae_fullrank_output.zip {CONFIG['outdir']}

from google.colab import files
files.download('/content/gppvae_fullrank_output.zip')
print("✅ Download started!")